In [2]:
import numpy as np
import scipy as sc
import pylab as py
import gzip
import math
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b, fmin_cg
from collections import defaultdict
import heapq
import copy
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import operator
import csv
import hashlib
from math import exp
from math import log
import bigfloat
import warnings
warnings.filterwarnings("error")
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random

In [39]:
data = []
with open('500 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], row['rating'], row['time']))

In [40]:
print len(data), data[0]

8241 ('A13G1TKIKHGV3F', '0060518499', '4.0', '1237075200')


In [41]:
itemsuserrated = defaultdict(lambda: list())
useritems = defaultdict(lambda: list())
for d in data:
    itemsuserrated[d[0]].append((d[1], d[3]))

for u in itemsuserrated:
    items = itemsuserrated[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    itemsuserrated[u] = isorted
    useritems[u] = [i[0] for i in isorted]

In [42]:
ps = set()
for d in data:
    ps.add(d[1])
with open('500 core subset - with items not rated.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['user','irated','inrated'])
    writer.writeheader()
    for user in useritems:
        itemsrated = useritems[user] 
        for i in itemsrated:
            notrated = random.sample(ps.difference(itemsrated),1)
            for n in notrated:
                writer.writerow({'user': user, 'irated': i, 'inrated': n})

In [44]:
np.random.shuffle(data)
with open('500 core subset - with items not rated.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['user','irated','inrated'])
    writer.writeheader()
    for d in data:
        writer.writerow({'user': d[0], 'irated': d[1], 'inrated': d[2]})

In [45]:
data = []
with open('500 core subset - with items not rated.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['user'], row['irated'], row['inrated']))

In [46]:
print data[0], data[1]

('A2AYSFGUP5VTY3', '1856498050', '1401202209') ('A2OJW07GQRNJUT', 'B000K3B5W4', '0962558680')


In [47]:
ps = set()
us = set()
for d in data:
    ps.add(d[1])
    ps.add(d[2])
    us.add(d[0])

In [48]:
print len(us)
print len(ps)
print len(data)

10
7976
8241


In [49]:
users = {}
products = {}
for i,u in enumerate(us):
    users[u] = i
for j,p in enumerate(ps):
    products[p] = j

In [50]:
# # np.random.shuffle(data)
# dtest = set()
# dvalid= set() #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for r in data:
#     cr = len(useritems[r[0]])
#     vc = int((cr * 30)/float(100))
#     if dtestusers[r[0]]< vc:
#         dtestusers[r[0]] +=1
#         dtest.add(r)
#     if r not in dtest and dvalidusers[r[0]]<vc:
#         dvalidusers[r[0]]+=1
#         dvalid.add(r)

# dtrain = set()
# for r in data:
#     if r not in dtest and r not in dvalid:
#         dtrain.add(r)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [51]:
x60 = int((len(data) * 60) / float(100))
x20 = int((len(data) * 20) / float(100))
dtrain = data[:x60]
dvalid = data[x60: x60+ x20]
end =  x60+ x20
dtest = data[end:]
print len(dtrain), len(dvalid), len(dtest), len(data)
print len(dtrain) + len(dvalid) + len(dtest) == len(data)

4944 1648 1649 8241
True


In [52]:
print len(dtrain) + len(dvalid)+ len(dtest), len(data), len(set(data))

8241 8241 8241


In [53]:
print dtrain[0]

('A2AYSFGUP5VTY3', '1856498050', '1401202209')


In [54]:
#balanced error rate
def BER(predictions, labels):
    tp =0
    fp=0
    tn=0
    fn=0
    for i,l in enumerate(labels):
        if l==1 and predictions[i]==l:
            tp+=1
        if l==0 and predictions[i]==l:
            tn+=1
        if l==1 and predictions[i]==0:
            fn+=1
        if l==0 and predictions[i]==1:
            fp+=1
    fpr = fp / float(fp+tn)
    fnr = fn/ float(tp+fn)
    return 0.5 * (fpr + fnr)

In [55]:
#function that takes a dataset and the optimized parameters and the diminsionality of the factors and returns the predictions for
#this dataset using the parmaters
def getPredictions_score(data, op,k):
    predictions = []
    betaio = op[:len(products)]
    end = len(products)
    gs = np.array(op[end:])
    opp = gs.reshape(len(users)+len(products),k)
    gammus = opp[:len(users),:]
    gammis= opp[len(users):,:]
    for d in data:
        p1 = betaio[products[d[1]]] + gammus[users[d[0]]].dot(gammis[products[d[1]]])
        p2 = betaio[products[d[2]]] + gammus[users[d[0]]].dot(gammis[products[d[2]]])
        predictions.append(sigmoid(p1))
        predictions.append(sigmoid(p2))
    return np.array(predictions)
    

In [56]:
#function that takes a dataset and the optimized parameters and the diminsionality of the factors and returns the predictions for
#this dataset using the parmaters
def getPredictions(data, op,k):
    predictions = []
    betaio = op[:len(products)]
    end = len(products)
    gs = np.array(op[end:])
    opp = gs.reshape(len(users)+len(products),k)
    gammus = opp[:len(users),:]
    gammis= opp[len(users):,:]
    for d in data:
        predictions.append(sigmoid(betaio[products[d[1]]] + (gammus[users[d[0]]].dot(gammis[products[d[1]]])))>0.5)
        predictions.append(sigmoid(betaio[products[d[2]]]+ (gammus[users[d[0]]].dot(gammis[products[d[2]]])))>0.5)
    return np.array(predictions)
    

In [57]:
def getPrediction(d, op, k,f):
    betaio = op[:len(products)]
    end = len(products)
    gs = np.array(op[end:])
    opp = gs.reshape(len(users)+len(products),k)
    gammus = opp[:len(users),:]
    gammis= opp[len(users):,:]
    if f:
        item = d[1]
    else:
        item = d[2]
    prediction  = sigmoid(betaio[products[item]] + (gammus[users[d[0]]].dot(gammis[products[item]])))
    return prediction

In [58]:
def AUC(data, op, k):
    count = 0
    predictions = getPredictions_score(data, op,k)
    for i in range(0, len(predictions), 2):
        if predictions[i] > predictions[i+1]:
            count +=1
#     print count, len(data)
    return count / float(len(data))     

In [59]:
# print AUC(dtest, params, 10)

In [60]:
#takes the data and returns the true labels for this dataset
def getLabels(data):
    labels = []
    for d in data:
        labels.append(1)
        labels.append(0)
    return labels

In [61]:
def sigmoid(z):
    try:
        f = sc.special.expit(z)
    except RuntimeWarning:
        f = 1/float(1 + bigfloat.exp(-z, bigfloat.precision(100)))
    return f

In [62]:
uintrain = set()
pintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    pintrain.add(d[2])
    uintrain.add(d[0])

In [63]:
def compute_cost(initial,lam,k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    gammas = np.array(initial[end:])
    gs = gammas.reshape(len(users)+len(products), k)
    gammau = gs[:len(users),:]
    gammai = gs[len(users):, :]
    J = 0
    for d in dataset: 
        term = (betai[products[d[1]]] + gammau[users[d[0]]].dot(gammai[products[d[1]]])) - (betai[products[d[2]]] + gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        J+= np.logaddexp(0, -term)
    for u in us:
        J += lam * np.sum(np.square(gammau[users[u]]))
    for p in ps:
        J+= lam * (np.square(betai[products[p]]) + np.sum(np.square(gammai[products[p]])))

    g = gradient(betai,gammau,gammai,lam,k, dataset)
#     print cost(initial, k , dtest)
    return J,g

In [64]:
def gradient(betai,gammau,gammai, lam,k, dataset):
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    
    for d in dataset:
        term = (betai[products[d[1]]] + gammau[users[d[0]]].dot(gammai[products[d[1]]])) - (betai[products[d[2]]] + gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        try:
            f = np.exp(-term) * sigmoid(term)
        except RuntimeWarning:
            f = float((bigfloat.exp(-term, bigfloat.precision(100))) * (sigmoid(term)))
     
        ggu[users[d[0]]] += (gammai[products[d[2]]] - gammai[products[d[1]]]) * f
        ggi[products[d[1]]]  += (-gammau[users[d[0]]]) * f
        ggi[products[d[2]]] += gammau[users[d[0]]] * f
        gbi[products[d[1]]] += (-1) * f
        gbi[products[d[2]]] += f

    for u in us:
        ggu[users[u]] += 2 * lam * gammau[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam * betai[products[p]]
        ggi[products[p]] += 2 * lam * gammai[products[p]]
    
    l = list(gbi) + list(ggu.flatten()) + list(ggi.flatten())

    return np.array(l)    

In [65]:
def cost(initial,k, dataset):
    betai = np.array(initial[:len(products)])
    end = len(products)
    gammas = np.array(initial[end:])
    gss = gammas.reshape(len(users)+len(products), k)
    gammau = gss[:len(users),:]
    gammai = gss[len(users):, :]
    J = 0
    for d in dataset: 
        term = (betai[products[d[1]]] +gammau[users[d[0]]].dot(gammai[products[d[1]]]))-(betai[products[d[2]]] +gammau[users[d[0]]].dot(gammai[products[d[2]]]))
        J+= np.logaddexp(0, -term)
   
    return J

In [66]:
class ConvergedException(Exception):
    pass
def check(xk):
    global result
    global presult
    global maxauc
    global count
    global mincost
    global nit
    
    nit+=1

    ccost = cost(xk, k, dvalid)
    
    if ccost> mincost:
        if nit >=100:
            count +=1
            if count == 20:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
#         print str(nit)  + ': yes'
        mincost = ccost
        presult[:] = xk
        count = 0
#     print ccost, mincost

#     fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, xk, k))
#     cauc = auc(fpr,tpr)
#     if cauc < maxauc and nit>=100:
#         count +=1
#         if count == 10:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#     else: 
#         maxauc = cauc
#         count = 0
#         presult[:] = xk
#     print cauc, maxauc, presult[0]
    

In [67]:
# class ConvergedException(Exception):
#     pass
# def check_updated(xk):
#     global result
#     global presult
#     global maxauc
#     global count
#     global mincost
#     global nit
#     global counter
#     global pcost
    
#     count+=1
#     nit+=1
#     if count==5:
#         count = 0
#         ccost = cost(xk, k, dvalid)
#         print ccost
#         if ccost> pcost:
#             result[:] = presult
#             raise ConvergedException()
#             return
#         else:
#             pcost = ccost
#             presult[:] = xk


In [68]:
# class ConvergedException(Exception):
#     pass
# def check2(xk):
#     global result
#     global presult
#     global maxauc
#     global count
#     global mincost
#     global nit
    
#     nit+=1

#     ccost = cost(xk, k, dvalid)
    
#     if ccost <  mincost:
#         mincost = ccost
#         print str(nit) + ' yes'


In [69]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global presult
    global maxauc
    global count
    global mincost
    global nit
    
    nit+=1

#     ccost = cost(xk, k, dvalid)
    
#     if ccost> mincost:
#         count +=1
#         if count == 30:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#             return

#     else:
#         mincost = ccost
#         presult[:] = xk
#         count = 0
#     fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, xk, k))
    cauc = AUC(dvalid, xk, k)
    if cauc < maxauc:
        count +=1
        if count == 30:
            print 'here'
            result[:] = presult
            raise ConvergedException()
    else: 
        maxauc = cauc
        count = 0
        presult[:] = xk

In [70]:
k = 10
betai = np.random.normal(0,0.1, len(products))
gammai = np.random.normal(0, 0.1, (len(products), k))
gammau = np.random.normal(0, 0.1, (len(users), k))
for u in users:
    if u not in uintrain:
        gammau[users[u]] = 0
for p in products:
    if p not in pintrain:
        gammai[products[p]]=0
        betai[products[p]]=0
initial = list(betai) + list(gammau.flatten()) + list(gammai.flatten())

In [71]:
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for l in lams:
    count = 0
    presult = []
    result = []
    params = []
    maxauc = 0
    mincost = 100000000000000
    nit = 0
    try:
        params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l,k, dtrain, ), disp = 0, callback=check_updated)
        params = presult
        print 'normal'
    except ConvergedException:
        params = result
    print 'converged after :' + str(nit)
    fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params, k))
#     fprt, tprt, _ = roc_curve(getLabels(dtest),getPredictions_score(dtest, params, k))
    print str(l)  + ' : ' + str(cost(params, k, dvalid)) + ' ' +  str(cost(params, k , dtest)) + ' ' + str(AUC(dvalid, params, k)) + ' ' + str(AUC(dtest, params, k))

normal
converged after :21
0 : 5595.40401234 5508.93360091 0.429004854369 0.432989690722
normal
converged after :69
1e-05 : 1076.10032642 1070.0161142 0.62682038835 0.616737416616
normal
converged after :96
0.0001 : 969.034076799 951.351724125 0.64927184466 0.637962401455
normal
converged after :113
0.001 : 950.448515044 937.821694377 0.648058252427 0.630685263796
normal
converged after :125
0.01 : 934.740161747 926.915878584 0.646844660194 0.62947240752
here
converged after :94
0.1 : 940.572937296 937.54995637 0.640776699029 0.624014554275
here
converged after :62
1 : 986.389634101 986.131383916 0.62682038835 0.605821710127
normal
converged after :23
10 : 1146.76797416 1144.45765989 0.45145631068 0.436628259551
normal
converged after :5
100 : 1144.73452351 1142.58344622 0.457524271845 0.453001819284
normal
converged after :4
1000 : 1144.53120496 1142.39588973 0.455703883495 0.453001819284
normal
converged after :4
10000 : 1144.51088127 1142.37714008 0.455703883495 0.453001819284


In [429]:
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
for l in lams:
    count = 0
    presult = []
    result = []
    params = []
    maxauc = 0
    mincost = 100000000000000
    nit = 0
    try:
        params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l,k, dtrain, ), disp = 0, callback=check_updated)
        params = presult
        print 'normal'
    except ConvergedException:
        params = result
    print 'converged after :' + str(nit)
    fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params, k))
    fprt, tprt, _ = roc_curve(getLabels(dtest),getPredictions_score(dtest, params, k))
    print str(l)  + ' : ' + str(cost(params, k, dvalid)) + ' ' +  str(auc(fpr,tpr))  + ' ' +  str(auc(fprt, tprt))

normal
converged after :29
0 : 10722.1570805 0.908940698818 0.909441614208
here
converged after :219
1e-05 : 13.6294679753 0.999788428508 0.999759809201
normal
converged after :227
0.0001 : 10.2761326616 0.999907260121 0.999883413142
normal
converged after :253
0.001 : 12.7350051074 0.999921290096 0.99989877761
normal
converged after :257
0.01 : 28.118073845 0.999929523306 0.999911967505
here
converged after :193
0.1 : 110.634852952 0.999935607082 0.999921656715
here
converged after :151
1 : 590.379695884 0.999933838933 0.999925932537
here
converged after :112
10 : 3660.69079559 0.999796516118 0.999813592172
normal
converged after :12
100 : 15181.7568694 0.501763007776 0.501902232818
normal
converged after :5
1000 : 15183.9386049 0.501215771677 0.501096545164
normal
converged after :4
10000 : 15184.1623682 0.501161244994 0.501017703278


In [288]:
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
with open('Binary baseline - gamma and beta terms 400 core.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda','k','value'])
    writer.writeheader()
    for l in lams:
        maxauc = 0
        count = 0
        presult = []
        result = []
        params = []
        nit = 0
        mincost = 100000000
        try:
            params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l,k, dtrain, ), disp = 0, callback=check)
            params = presult
        except ConvergedException:
            params = presult
        print 'converged after :' + str(nit)
        fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params, k))
        print str(l)  + ' : ' +  str(auc(fpr,tpr))
#         print cost(params, k, dvalid)
        for i, v in enumerate(params):
            writer.writerow({'lambda': l, 'k': k, 'value': v}) 

converged after :26
0 : 0.877802970049
here
converged after :162
1e-05 : 0.999806885144
here
converged after :131
0.0001 : 0.999820241589
here
converged after :171
0.001 : 0.999875918407
here
converged after :153
0.01 : 0.99989908138
here
converged after :247
0.1 : 0.999899152609
here
converged after :119
1 : 0.999231904386
here
converged after :119
10 : 0.989200077772
converged after :13
100 : 0.496867983336
converged after :5
1000 : 0.475989757711
converged after :4
10000 : 0.476459606876


In [289]:
params = defaultdict(lambda:defaultdict(list))
with open('Binary baseline - gamma and beta terms 400 core.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        params[float(row['lambda'])][int(row['k'])].append(float(row['value']))

In [290]:
print len(params)

11


In [291]:
#takes dictionary of parameters in the form of  {lambda value: k value: [parameters]} and choose the lambda value and k value that
#minimize the balanced error rate
def chooseBestParams(data, params):
    minError = 1000000000
    maxauc = 0
    labs = getLabels(data)
    for l in params:
        for k in params[l]:
#             fpr, tpr, _ = roc_curve(getLabels(dvalid),getPredictions_score(dvalid, params[l][k], k))
#             cauc = auc(fpr,tpr)
#             if cauc > maxauc:
#                 maxauc = cauc
#                 minlam = l
#                 op = params[l][k]


            e = cost(params[l][k], k, data)
#             print l, e
#             print l, k , e
            if e < minError:
                minError = e
                minlam = l
                op = params[l][k]
    return minlam, op
# , minError

In [292]:
minlam, op = chooseBestParams(dvalid, params)
print minlam

0.001


In [293]:
print cost(op,k, dtest)

11.4094030928


In [294]:
fpr, tpr, _ = roc_curve(getLabels(dtest),getPredictions_score(dtest, op, k))
roc_auc = auc(fpr,tpr)
print roc_auc

0.999895980833


In [295]:
print cost(op,k, dtrain)

0.740742461372


In [296]:
fpr, tpr, _ = roc_curve(getLabels(dtrain),getPredictions_score(dtrain, op, k))
roc_auc = auc(fpr,tpr)
print roc_auc

1.0


In [297]:
preds = getPredictions(dtest, op, k)
labels = getLabels(dtest)
print BER(preds, labels)

0.154293299158


In [209]:
# fpr, tpr, _ = roc_curve(getLabels(dtest),getPredictions_score(dtest, op, mink))
# roc_auc = auc(fpr,tpr)
# print roc_auc
# fpr, tpr, _ = roc_curve(getLabels(dtrain),getPredictions_score(dtrain, op, mink))
# roc_auc = auc(fpr,tpr)
# print roc_auc
# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.5f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve gamma terms and beta terms')
# plt.legend(loc="lower right")
# plt.show()

In [29]:
params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(0,10, dtrain, ))

In [30]:
print compute_cost(params, 0, 10, dtrain)

(1.6063525076659194e-05, array([ -9.60714517e-09,  -3.18075805e-09,  -5.48235994e-10, ...,
        -3.74857351e-09,  -3.69609469e-09,   5.90983072e-09]))
